### Generate HRU at different complexity levels ###

In [1]:
import os
import numpy as np
import rasterio as rio
import geopandas as gpd
import geospatial_functions.utils as ut
import geospatial_functions.geospatial_analysis as ga
import matplotlib.pyplot as plt 

In [2]:
# common paths
control_file = 'control_active.txt'
root_path = ut.read_from_control(control_file, 'root_path')
source_path = ut.read_from_control(control_file, 'source_path')
domain_name = ut.read_from_control(control_file, 'domain_name')
domain_path = os.path.join(root_path, domain_name)

In [3]:
# domain data
domain_gru_prj_shp = ut.specify_file_path(control_file, 'domain_gru_prj_shp')
domain_gru_raster = ut.specify_file_path(control_file, 'domain_gru_raster')
gruNo_field = ut.read_from_control(control_file, 'gruNo_field')
gruNo_field_dtype = ut.read_from_control(control_file, 'gruNo_field_dtype')
gruName_field = ut.read_from_control(control_file, 'gruName_field')
domain_gru_corr_txt = ut.specify_file_path(control_file, 'domain_gru_corr_txt')

domain_dem_raster = ut.specify_file_path(control_file, 'domain_dem_raster')  
domain_slope_raster = ut.specify_file_path(control_file, 'domain_slope_raster')  
domain_aspect_raster = ut.specify_file_path(control_file, 'domain_aspect_raster')
domain_landcover_class_raster = ut.specify_file_path(control_file, 'domain_landcover_class_raster')
domain_soil_raster = ut.specify_file_path(control_file, 'domain_soil_raster')
domain_radiation_raster = ut.specify_file_path(control_file, 'domain_radiation_raster')

refraster = ut.specify_refraster_path(control_file)

In [4]:
# hru data
hruNo_field = ut.read_from_control(control_file, 'hruNo_field')
hruNo_field_dtype = ut.read_from_control(control_file,'hruNo_field_dtype') # used to save hruNo raster. 
hruName_field = ut.read_from_control(control_file,'hruName_field')         # field name of hru name, e.g., 10080012010101, 100800120102. 
    
elev_class_field = ut.read_from_control(control_file,'elev_class_field')           # field name of the elevation class column in HRU. 
land_class_field = ut.read_from_control(control_file,'land_class_field')           # field name of the land class column in HRU. 
radiation_class_field = ut.read_from_control(control_file,'radiation_class_field') # field name of the radiation class column in HRU. 
hru_area_field =  = ut.read_from_control(control_file,'hru_area_field')            # field name of the HRU area.

hru_thld_type = ut.read_from_control(control_file,'hru_thld_type')         # use a fraction or area value to eliminate small HRUs.
hru_thld = float(ut.read_from_control(control_file, 'hru_thld'))           # if hru_thld_type = 'fraction', hru_thld = partial of the gru area.
                                                                           ## if hru_thld_type = 'value', hru_thld = elimination area.

In [5]:
# basename for dem and radiation classification in a hru level.
dem_class_basename = 'dem_class' # basename for DEM class files (eg, 0:low elevation. 1: high elevation).
dem_value_basename = 'dem_value' # basename for DEM value files (eg, average DEM per class).

rad_class_basename = 'rad_class' # basename for radiation class files (eg, 0:low. 1:high).
rad_value_basename = 'rad_value' # basename for radiation value files (eg, average radiation per class).

#### Define HRU complexity levels ####
level 0: GRU = HRU. <br>
level 1: use only elevation bands in HRU generation.<br>
level 2a: use elevation bands and landcover classes in HRU generation.<br>
level 2b: use elevation bands and radiation classes in HRU generation.<br>
level 3a: use elevation bands, radiation bands, landcover classes in HRU generation.<br>
level 3b: nested. radiation bands are generated based on level 2a HRU.<br>

In [9]:
level_list = ['0','1','2a','2b','3a','3b']

#### 1. Discretize HRU ####

In [7]:
for level in level_list:

    print('--- Complexity level %s ---' %(level))
    
    #  --- PART 1. define output files of discretization--- 
    hru_str = 'hru'+'_lev' + str(level)
    hru_elmn_str = hru_str+'_elmn'     
    
    hru_raster = os.path.join(domain_path, hru_str+'.tif')            # original HRU
    hru_vector = os.path.join(domain_path, hru_str+'.shp')
    hru_raster_elmn = os.path.join(domain_path, hru_elmn_str+'.tif')  # simplified HRU
    hru_vector_elmn = os.path.join(domain_path, hru_elmn_str+'.shp')    

    dem_classif_trigger = 300 # Elvation difference value per GRU used to trigger elevation classification.
    dem_bins = 'median' # Elevation classification method. 'median' means using the median value per GRU as the classification threhold.
    dem_class_raster = os.path.join(domain_path, dem_class_basename+'_lev'+str(level)+'.tif')
    dem_value_raster = os.path.join(domain_path, dem_value_basename+'_lev'+str(level)+'.tif')
    
    rad_classif_trigger = 50 #None # Radiation difference value per GRU used to trigger elevation classification.
    rad_bins = 'median' # Radiation classification method. 'median' means using the median value per GRU as the classification threhold.
    rad_class_raster = os.path.join(domain_path, rad_class_basename+'_lev'+str(level)+'.tif')
    rad_value_raster = os.path.join(domain_path, rad_value_basename+'_lev'+str(level)+'.tif')
    
    #  --- PART 2. define inputs of discretization ---
    # raster_list: a list of raster inputs that are used to define HRU.
    # fieldname_list: a list of field names corresponding to raster_list.
    
    # level 0: GRU = HRU (benchmark). 
    if level == '0': 
        # (1) define input files for hru discretization
        raster_list = [domain_gru_raster]
        fieldname_list = [gruNo_field]  

    # level 1: use only elevation bands in HRU generation.
    if level == '1': 
        # (1) classify elevation raster per gru
        ga.classify_raster(domain_dem_raster, domain_gru_raster, dem_classif_trigger, dem_bins,
                           dem_class_raster, dem_value_raster)        
        # (2) define input files for hru discretization
        raster_list = [domain_gru_raster,dem_class_raster]
        fieldname_list = [gruNo_field, elev_class_field]
    
    # level 2a: use elevation bands and landcover classes in HRU generation.
    elif level == '2a': 
        # (1) classify elevation raster per gru
        ga.classify_raster(domain_dem_raster, domain_gru_raster, dem_classif_trigger, dem_bins,
                           dem_class_raster, dem_value_raster)        
        # (2) define input files for hru discretization
        raster_list = [domain_gru_raster, dem_class_raster, domain_landcover_class_raster]
        fieldname_list = [gruNo_field, elev_class_field, land_class_field]

    # level 2b: use elevation bands and radiation classes in HRU generation.
    elif level == '2b': 
        # (1) classify elevation raster per gru
        ga.classify_raster(domain_dem_raster, domain_gru_raster, dem_classif_trigger, dem_bins,
                           dem_class_raster, dem_value_raster)        
        # (2) classify radiation raster per gru
        ga.classify_raster(domain_radiation_raster, domain_gru_raster, rad_classif_trigger, rad_bins, 
                           rad_class_raster, rad_value_raster)        
        # (3) define input files for hru discretization
        raster_list = [domain_gru_raster, dem_class_raster, rad_class_raster]
        fieldname_list = [gruNo_field, elev_class_field, radiation_class_field]
    
    
    # level 3a: use elevation bands, radiation bands, landcover classes in HRU generation.
    elif level == '3a':
        # (1) classify elevation raster per gru
        ga.classify_raster(domain_dem_raster, domain_gru_raster, dem_classif_trigger, dem_bins,
                           dem_class_raster, dem_value_raster)        
        # (2) classify radiation raster per gru
        ga.classify_raster(domain_radiation_raster, domain_gru_raster, rad_classif_trigger, rad_bins, 
                           rad_class_raster, rad_value_raster)        
        # (3) define input files for hru discretization
        raster_list = [domain_gru_raster, dem_class_raster, rad_class_raster, domain_landcover_class_raster]
        fieldname_list = [gruNo_field, elev_class_field, radiation_class_field, land_class_field]

    # level 3b: nested. radiation bands are generated based on level 2a HRU.
    elif level == '3b':
        # (1) classify elevation raster per gru
        ga.classify_raster(domain_dem_raster, domain_gru_raster, dem_classif_trigger, dem_bins, 
                                      dem_class_raster, dem_value_raster)        
        # (2) classify radiation raster per level 2a hru
        hru_lev2a_raster = os.path.join(domain_path, 'hru'+'_lev2a_elmn.tif')
        ga.classify_raster(domain_radiation_raster, hru_lev2a_raster, rad_classif_trigger, rad_bins, 
                                      rad_class_raster, rad_value_raster)        
        # (3) define input files for hru discretization
        raster_list = [domain_gru_raster, dem_class_raster, rad_class_raster, domain_landcover_class_raster]
        fieldname_list = [gruNo_field, elev_class_field, radiation_class_field, land_class_field]

    # --- PART 3. genearte HRU based on gru and elevation class ---
    ga.define_hru(raster_list, fieldname_list, domain_gru_raster, domain_gru_corr_txt, gruNo_field, gruName_field,
                  hru_raster, hru_vector, hruNo_field, hruNo_field_dtype, hruName_field)

    # --- PART 4. calculate HRU area ---
    in_gpd = gpd.read_file(hru_vector)
    in_gpd[hru_area_field] = in_gpd.area
    in_gpd.to_file(hru_vector)

    # --- PART 5. eliminate small area HRUs ---
    # mehod 1: change HRU attribute to the most dominant HRU within the GRU. Use function ga.eliminate_small_hrus_dominant
    # method 2: change HRU attribute to its' largest neighbor's HRU
    ga.eliminate_small_hrus_neighbor(hru_vector, hru_thld_type, hru_thld, gruNo_field, gruName_field, 
                                     hruNo_field, hruNo_field_dtype, hruName_field, hru_area_field, 
                                     fieldname_list, refraster, 
                                     hru_vector_elmn, hru_raster_elmn)
#     ga.plot_vector(hru_vector_elmn, hruName_field) # quick plot for check

--- Complexity level 2b ---


100%|██████████| 18/18 [00:00<00:00, 51.03it/s]


#### 2. Calculate HRU zonal statistics ####

In [8]:
for level in level_list:

    print('--- Complexity level %s ---' %(level))
    
    #  --- PART 1. define hru complexity level dependent files --- 
    hru_str = 'hru'+'_lev' + str(level)
    hru_elmn_str = hru_str+'_elmn'     
    
    hru_vector = os.path.join(domain_path, hru_str+'.shp')
    hru_vector_elmn = os.path.join(domain_path, hru_elmn_str+'.shp')    
    
    # --- PART 2. calculate zonal statistics ---
    for invector in [hru_vector,hru_vector_elmn]:
        
        # (1) define invector dependent files 
        invector_field = hruNo_field
        invector_field_dtype = hruNo_field_dtype
        
        attrb_elev = os.path.join(domain_path, os.path.basename(invector).split('.shp')[0]+'_attrb_elevation.tif')        
        attrb_slp = os.path.join(domain_path, os.path.basename(invector).split('.shp')[0]+'_attrb_slope.tif')        
        attrb_asp = os.path.join(domain_path, os.path.basename(invector).split('.shp')[0]+'_attrb_aspect.tif')        
        attrb_lc = os.path.join(domain_path, os.path.basename(invector).split('.shp')[0]+'_attrb_landcover.tif')        
        attrb_soil = os.path.join(domain_path, os.path.basename(invector).split('.shp')[0]+'_attrb_soil.tif')        
        
        # (2) elevation zonal statistics 
        ga.zonal_statistic(domain_dem_raster, invector, invector_field, invector_field_dtype, 
                           refraster, 'mean', attrb_elev, output_column_prefix='elev')

        # (3) slope zonal statistics 
        ga.zonal_statistic(domain_slope_raster, invector, invector_field, invector_field_dtype, refraster, 
                           'mean', attrb_slp, output_column_prefix='slope')

        # (4) aspect zonal statistics 
        ga.zonal_statistic(domain_aspect_raster, invector, invector_field, invector_field_dtype, refraster, 
                           'mean_aspect', attrb_asp, output_column_prefix='aspect')

        # (5) landcover zonal statistics 
        ga.zonal_statistic(domain_landcover_class_raster, invector, invector_field, invector_field_dtype, refraster, 
                           'mode', attrb_lc, output_column_prefix='vegType')        
        
        # (6) soil zonal statistics 
        ga.zonal_statistic(domain_soil_raster, invector, invector_field, invector_field_dtype, refraster, 
                           'mode', attrb_soil, output_column_prefix='soilType')        
        
        # -------- post-process attributes for SUMMA ---------
        # (7) landcover and soil types 
        # convert landcover int to [1,17] range 
        # change soilType from float to int (because source soilType is float)
        in_gpd = gpd.read_file(invector)
        in_gpd['vegType'] = in_gpd['vegType']+1
        in_gpd['soilType'] = in_gpd['soilType'].astype('int')
        in_gpd.to_file(invector)
        
        # (8) convert landcover int to string for easy understanding
        lcClass_list = [ 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 255]
        lcValue_list = ['Evergreen needleleaf forests', 'Evergreen broadleaf forests', 'Deciduous needleleaf forests',
                        'Deciduous broadleaf forests', 'Mixed forests', 'Closed shrublands', 'Open shrublands', 
                        'Woody savannas', 'Savannas', 'Grasslands', 'Permanent wetlands', 'Croplands', 
                        'Urban and built-up lands', 'Cropland/natural vegetation mosaics', 'Snow and ice', 
                        'Barren', 'Water bodies', 'None']
        in_gpd = gpd.read_file(invector)
        in_gpd['landcover'] = ""
        for irow, row in in_gpd.iterrows():
            lcClass = in_gpd.loc[irow,'vegType'] 
            lcValue=lcValue_list[lcClass_list.index(lcClass)]
            in_gpd.at[irow,'landcover'] = lcValue
        in_gpd['landcover'] = in_gpd['landcover'].astype('str')
        in_gpd.to_file(invector)

        # (9) convert ROSETTA soil to STAS and add string for easy understanding
        soilClass_list = [ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
        soilValue_list = ['OTHER(land-ice)', 'CLAY', 'CLAY LOAM', 'LOAM', 'LOAMY SAND', 'SAND', 'SANDY CLAY', 
                          'SANDY CLAY LOAM', 'SANDY LOAM', 'SILT','SILTY CLAY', 'SILTY CLAY LOAM', 'SILT LOAM']
        soilClass_list_STAS = [16, 12, 9, 6, 2, 1, 10, 7, 3, 5, 11, 8, 4]
        
        in_gpd = gpd.read_file(invector)
        in_gpd['soilROSETTA'] = in_gpd['soilType']
        in_gpd['soilSTAS'] = ""
        in_gpd['soil'] = ""
        for irow, row in in_gpd.iterrows():
            
            soilClass = in_gpd.loc[irow,'soilType'] 
            if soilClass==0:
                lcClass = in_gpd.loc[irow,'vegType'] 
                print('hruNo = %d, soilType_ROSETTA = 0, and vegType = %s.'%(in_gpd.loc[irow,'hruNo'],lcClass))
            
            soilValue=soilValue_list[soilClass_list.index(soilClass)]
            soilClass_STAS=soilClass_list_STAS[soilClass_list.index(soilClass)]
            
            in_gpd.at[irow,'soil'] = soilValue
            in_gpd.at[irow,'soilSTAS'] = soilClass_STAS
            
        in_gpd['soil'] = in_gpd['soil'].astype('str')
        in_gpd['soilSTAS'] = in_gpd['soilSTAS'].astype('int')
        in_gpd['soilType'] = in_gpd['soilSTAS']
        in_gpd = in_gpd.drop(columns=['soilSTAS'])
        in_gpd.to_file(invector)

        # (10) convert slope to tan_slope 
        in_gpd = gpd.read_file(invector)
        in_gpd['tan_slope'] = np.tan(np.radians(in_gpd['slope']))
        in_gpd.to_file(invector)

        # (11) calculate contourLength (meter)
        # assuming the hru area is a circle and taking the radius as contourLength.
        in_gpd = gpd.read_file(invector)
        in_gpd['contourLength'] = np.power(in_gpd['areaSqm']/np.pi,0.5)
        in_gpd.to_file(invector)

        # (12) calculate centroid lat/lon (degree)
        def getXY(pt):
            return (pt.x, pt.y)
        in_gpd = gpd.read_file(invector)
        in_gpd_wgs84 = in_gpd.copy()
        in_gpd_wgs84 = in_gpd_wgs84.to_crs(epsg=4326) #"EPSG:4326"
        centroidseries = in_gpd_wgs84['geometry'].centroid
        in_gpd['longitude'],in_gpd['latitude']=[list(t) for t in zip(*map(getXY, centroidseries))]
        in_gpd.to_file(invector)

    # --- PART 3. save HRU with attributes into gpkg ---
    invector_gpkg = os.path.join(domain_path, os.path.basename(invector).split('.shp')[0]+'.gpkg')
    in_gpd = gpd.read_file(invector)
    in_gpd.to_file(invector_gpkg, driver="GPKG")


--- Complexity level 2b ---


/Users/hongliliu/Documents/github/discretization/discretize/geospatial_functions/geospatial_analysis.py:920: RuntimeWarning: invalid value encountered in sin
  y = np.sin(np.radians(attr_smask)) # north/south vector (positive to N)
/Users/hongliliu/Documents/github/discretization/discretize/geospatial_functions/geospatial_analysis.py:921: RuntimeWarning: invalid value encountered in cos
  x = np.cos(np.radians(attr_smask)) # west/east vector (positive to E)
/Users/hongliliu/Documents/github/discretization/discretize/geospatial_functions/geospatial_analysis.py:920: RuntimeWarning: invalid value encountered in sin
  y = np.sin(np.radians(attr_smask)) # north/south vector (positive to N)
/Users/hongliliu/Documents/github/discretization/discretize/geospatial_functions/geospatial_analysis.py:921: RuntimeWarning: invalid value encountered in cos
  x = np.cos(np.radians(attr_smask)) # west/east vector (positive to E)
<ipython-input-8-5cfe523697e9>:97: UserWarning: Column names longer than 10 

#### 3. Calculate GRU zonal statistics (optional) ####

In [14]:
invector = domain_gru_prj_shp
invector_field = gruNo_field
invector_field_dtype = gruNo_field_dtype

attrb_elev = os.path.join(domain_path, os.path.basename(invector).split('.shp')[0]+'_attrb_elevation.tif')        
attrb_slp = os.path.join(domain_path, os.path.basename(invector).split('.shp')[0]+'_attrb_slope.tif')        
attrb_asp = os.path.join(domain_path, os.path.basename(invector).split('.shp')[0]+'_attrb_aspect.tif')        
attrb_lc = os.path.join(domain_path, os.path.basename(invector).split('.shp')[0]+'_attrb_landcover.tif')        
attrb_soil = os.path.join(domain_path, os.path.basename(invector).split('.shp')[0]+'_attrb_soil.tif')        

outvector = os.path.join(domain_path, os.path.basename(invector).split('.shp')[0]+'_attrb.gpkg')     
in_gpd = gpd.read_file(invector)
in_gpd.to_file(outvector, driver="GPKG")
invector = outvector # avoid process gru_shp_prj. Work on gpkg.

# (1) calculate zonal area ---
in_gpd = gpd.read_file(invector)
in_gpd['areaSqm'] = in_gpd.area
in_gpd.to_file(invector)

# (2) elevation zonal statistics 
ga.zonal_statistic(domain_dem_raster, invector, invector_field, invector_field_dtype, refraster, 'mean', attrb_elev,
                   output_column_prefix='elev')

# (3) slope zonal statistics 
ga.zonal_statistic(domain_slope_raster, invector, invector_field, invector_field_dtype, refraster, 'mean', attrb_slp,
                   output_column_prefix='slope')

# (4) aspect zonal statistics 
ga.zonal_statistic(domain_aspect_raster, invector, invector_field, invector_field_dtype, refraster, 'mean_aspect', attrb_asp,
                   output_column_prefix='aspect')

# (5) landcover zonal statistics 
ga.zonal_statistic(domain_landcover_class_raster, invector, invector_field, invector_field_dtype, refraster, 'mode', attrb_lc,
                   output_column_prefix='vegType')        

# (6) soil zonal statistics 
ga.zonal_statistic(domain_soil_raster, invector, invector_field, invector_field_dtype, refraster, 'mode', attrb_soil,
                   output_column_prefix='soilType')        

# -------- post-process attributes for SUMMA ---------
# (7) landcover and soil types 
# convert landcover int to [1,17] range 
# change soilType from float to int (because source soilType is float)
in_gpd = gpd.read_file(invector)
in_gpd['vegType'] = in_gpd['vegType']+1
in_gpd['soilType'] = in_gpd['soilType'].astype('int')
in_gpd.to_file(invector)

# (8) convert landcover int to string for easy understanding
lcClass_list = [ 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 255]
lcValue_list = ['Evergreen needleleaf forests', 'Evergreen broadleaf forests', 'Deciduous needleleaf forests',
                'Deciduous broadleaf forests', 'Mixed forests', 'Closed shrublands', 'Open shrublands', 
                'Woody savannas', 'Savannas', 'Grasslands', 'Permanent wetlands', 'Croplands', 
                'Urban and built-up lands', 'Cropland/natural vegetation mosaics', 'Snow and ice', 
                'Barren', 'Water bodies', 'None']
in_gpd = gpd.read_file(invector)
in_gpd['landcover'] = ""
for irow, row in in_gpd.iterrows():
    lcClass = in_gpd.loc[irow,'vegType'] 
    lcValue=lcValue_list[lcClass_list.index(lcClass)]
    in_gpd.at[irow,'landcover'] = lcValue
in_gpd['landcover'] = in_gpd['landcover'].astype('str')
in_gpd.to_file(invector)

# (9) convert ROSETTA soil to STAS and add string for easy understanding
soilClass_list = [ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
soilValue_list = ['OTHER(land-ice)', 'CLAY', 'CLAY LOAM', 'LOAM', 'LOAMY SAND', 'SAND', 'SANDY CLAY', 
                  'SANDY CLAY LOAM', 'SANDY LOAM', 'SILT','SILTY CLAY', 'SILTY CLAY LOAM', 'SILT LOAM']
soilClass_list_STAS = [16, 12, 9, 6, 2, 1, 10, 7, 3, 5, 11, 8, 4]

in_gpd = gpd.read_file(invector)
in_gpd['soilROSETTA'] = in_gpd['soilType']
in_gpd['soilSTAS'] = ""
in_gpd['soil'] = ""
for irow, row in in_gpd.iterrows():

    soilClass = in_gpd.loc[irow,'soilType'] 
    if soilClass==0:
        lcClass = in_gpd.loc[irow,'vegType'] 
        print('hruNo = %d, soilType_ROSETTA = 0, and vegType = %s.'%(in_gpd.loc[irow,'hruNo'],lcClass))

    soilValue=soilValue_list[soilClass_list.index(soilClass)]
    soilClass_STAS=soilClass_list_STAS[soilClass_list.index(soilClass)]

    in_gpd.at[irow,'soil'] = soilValue
    in_gpd.at[irow,'soilSTAS'] = soilClass_STAS

in_gpd['soil'] = in_gpd['soil'].astype('str')
in_gpd['soilSTAS'] = in_gpd['soilSTAS'].astype('int')
in_gpd['soilType'] = in_gpd['soilSTAS']
in_gpd = in_gpd.drop(columns=['soilSTAS'])
in_gpd.to_file(invector)

# (10) convert slope to tan_slope 
in_gpd = gpd.read_file(invector)
in_gpd['tan_slope'] = np.tan(np.radians(in_gpd['slope']))
in_gpd.to_file(invector)

# (11) calculate contourLength (meter)
# assuming the hru area is a circle and taking the radius as contourLength.
in_gpd = gpd.read_file(invector)
in_gpd['contourLength'] = np.power(in_gpd['areaSqm']/np.pi,0.5)
in_gpd.to_file(invector)

# (12) calculate centroid lat/lon (degree)
def getXY(pt):
    return (pt.x, pt.y)
in_gpd = gpd.read_file(invector)
in_gpd_wgs84 = in_gpd.copy()
in_gpd_wgs84 = in_gpd_wgs84.to_crs(epsg=4326) #"EPSG:4326"
centroidseries = in_gpd_wgs84['geometry'].centroid
in_gpd['longitude'],in_gpd['latitude']=[list(t) for t in zip(*map(getXY, centroidseries))]
in_gpd.to_file(invector)


/Users/hongliliu/Documents/github/discretization/discretize/geospatial_functions/geospatial_analysis.py:920: RuntimeWarning: invalid value encountered in sin
  y = np.sin(np.radians(attr_smask)) # north/south vector (positive to N)
/Users/hongliliu/Documents/github/discretization/discretize/geospatial_functions/geospatial_analysis.py:921: RuntimeWarning: invalid value encountered in cos
  x = np.cos(np.radians(attr_smask)) # west/east vector (positive to E)
/Users/hongliliu/Documents/github/discretization/discretize/geospatial_functions/geospatial_analysis.py:920: RuntimeWarning: invalid value encountered in sin
  y = np.sin(np.radians(attr_smask)) # north/south vector (positive to N)
/Users/hongliliu/Documents/github/discretization/discretize/geospatial_functions/geospatial_analysis.py:921: RuntimeWarning: invalid value encountered in cos
  x = np.cos(np.radians(attr_smask)) # west/east vector (positive to E)
<ipython-input-14-55840c6bcbb2>:93: UserWarning: Column names longer than 10

Done


<ipython-input-14-55840c6bcbb2>:112: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroidseries = in_gpd_wgs84['geometry'].centroid
<ipython-input-14-55840c6bcbb2>:114: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  in_gpd.to_file(invector)
